In [38]:
# Data Loading
import pandas as pd

df = pd.read_csv('loan_data_2007_2014.csv', index_col=None)
#removing index column
df.drop(['Unnamed: 0'], axis=1, inplace=True)

/var/folders/42/3d9r3rcx2vgd6bmrbmhv51k40000gn/T/ipykernel_57325/3902104741.py:4: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('loan_data_2007_2014.csv', index_col=None)


In [39]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Column Category

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466285 entries, 0 to 466284
Data columns (total 74 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           466285 non-null  int64  
 1   member_id                    466285 non-null  int64  
 2   loan_amnt                    466285 non-null  int64  
 3   funded_amnt                  466285 non-null  int64  
 4   funded_amnt_inv              466285 non-null  float64
 5   term                         466285 non-null  object 
 6   int_rate                     466285 non-null  float64
 7   installment                  466285 non-null  float64
 8   grade                        466285 non-null  object 
 9   sub_grade                    466285 non-null  object 
 10  emp_title                    438697 non-null  object 
 11  emp_length                   445277 non-null  object 
 12  home_ownership               466285 non-null  object 
 13 

## Numerical and categorical data
This step will helps to divide type of data for every columns

In [41]:
def getNumericalCategorical(x):
    '''
        Returns 2 list of columns for numerical data and categorical data.

                Parameters:
                        x (pandas DataFrame): DataFrame input to be inspected for the missing data
                Returns:
                        numCols (list): list of numerical columns from DataFrame x
                        catCols (list): list of cateforical columns from DataFrame x
    '''


    # getting columns list of numerical data from DataFrame x using _get_numeric_data() function
    numCols = x._get_numeric_data().columns.tolist()

    # determining cateforical columns by substracting all columns list by numCols
    catCols = list(set(x.columns) - set(numCols))

    return numCols, catCols

In [42]:
#numeric data
numCols, catCols = getNumericalCategorical(df)
print('there are',str(len(numCols)),'columns of numerical data. including:',str(numCols[:5]))
print('there are',str(len(catCols)),'columns of categorical data. including:',str(catCols[:5]))


there are 52 columns of numerical data. including: ['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv']
there are 22 columns of categorical data. including: ['application_type', 'emp_length', 'last_pymnt_d', 'pymnt_plan', 'grade']


### Notes

<b>52 columns</b> are having numerical data

<b>22 columns</b> are having categorical data


<br><br>
<div class="alert alert-block alert-success">
    <b> The data can be further divided into before and after loan application </b>
    <br>This is important to determine which columns will be used on credit risk assessment
</div>

# Missing Values

This step will help determining which columns are actively being used and the ones that never be used.

## Missing Values Function

In [43]:
def missing_cols(x):
        '''
        Returns a table in dataframe format.

                Parameters:
                        x (pandas DataFrame): DataFrame input to be inspected for the missing data
                Returns:
                        missing_value_df (pandas DataFrame): DataFrame output showing the missing data information for each columns in x (how many and its percentage)
        '''

        #acquiring column name, missing count and its percentage to the size of dataframe
        column_name =  x.columns
        missing_count = x.isnull().sum()
        percent_missing = missing_count * 100 / len(x)

        #creating a dataframe to store the missing row count
        missing_value_df = pd.DataFrame({
                                    'column_name': column_name,
                                    'missing_count' : missing_count,
                                    'percent_missing': percent_missing
                                    }).reset_index(drop=True)

        return missing_value_df



def missing_info(x):
        '''
        Returns 2 table in dataframe format.

                Parameters:
                        x (pandas DataFrame): DataFrame input to be inspected for the missing data
                Returns:
                        missing_value_df (pandas DataFrame): DataFrame output showing the missing data information for each columns in x (how many and its percentage)
                        missing_info (pandas DataFrame): DataFrame output showing a category of missing data (No Null, Semi Null, and fully null),
                                                                                how many columns for each category,
                                                                                and list of columns for each category
        '''
        missing_value_df = missing_cols(x)
        
        missing_category = ['No Null', 'Semi Null', 'Fully Null']
        
        NoNull_count = len(missing_value_df[missing_value_df['missing_count'] == 0])
        SemiNull_count = len(missing_value_df[(
                                        (missing_value_df['missing_count'] != 0) & 
                                        (missing_value_df['percent_missing'] != 100)
                                        )])
        FullyNull_count = len(missing_value_df[missing_value_df['percent_missing'] == 100])

        CountList = [NoNull_count, SemiNull_count, FullyNull_count]
        
        NoNull_tables = missing_value_df[missing_value_df['missing_count'] == 0]['column_name'].tolist()
        SemiNull_tables = missing_value_df[(
                                        (missing_value_df['missing_count'] != 0) & 
                                        (missing_value_df['percent_missing'] != 100)
                                        )]['column_name'].tolist()
        FullyNull_tables = missing_value_df[missing_value_df['percent_missing'] == 100]['column_name'].tolist()

        Tables = [NoNull_tables, SemiNull_tables, FullyNull_tables]

        missing_info = pd.DataFrame({
                                    'missing_category': missing_category,
                                    'table_number' : CountList,
                                    'table_list': Tables
                                    }).reset_index(drop=True)

        return missing_value_df, missing_info




## Missing Data Understanding

In [44]:
missings_cal, missing_stat = missing_info(df)
missings_cal

,column_name,missing_count,percent_missing
0,id,0,0.000000
1,member_id,0,0.000000
2,loan_amnt,0,0.000000
3,funded_amnt,0,0.000000
4,funded_amnt_inv,0,0.000000
...,...,...,...
69,all_util,466285,100.000000
70,total_rev_hi_lim,70276,15.071469
71,inq_fi,466285,100.000000
72,total_cu_tl,466285,100.000000


In [45]:
missing_stat

,missing_category,table_number,table_list
0,No Null,34,"[id, member_id, loan_amnt, funded_amnt, funded..."
1,Semi Null,23,"[emp_title, emp_length, annual_inc, desc, titl..."
2,Fully Null,17,"[annual_inc_joint, dti_joint, verification_sta..."


In [47]:
#showing only missing columns
missings_cal[missings_cal['missing_count'] != 0]

,column_name,missing_count,percent_missing
10,emp_title,27588,5.916553
11,emp_length,21008,4.505399
13,annual_inc,4,0.000858
19,desc,340302,72.981546
21,title,20,0.004289
25,delinq_2yrs,29,0.006219
26,earliest_cr_line,29,0.006219
27,inq_last_6mths,29,0.006219
28,mths_since_last_delinq,250351,53.690554
29,mths_since_last_record,403647,86.566585


In [36]:
#showing only missing columns and not entirely missing (100%)
missings_cal[((missings_cal['missing_count'] != 0) & (missings_cal['percent_missing'] != 100))].sort_values(by='percent_missing', ascending=False)

,column_name,missing_count,percent_missing
29,mths_since_last_record,403647,86.566585
50,mths_since_last_major_derog,367311,78.773926
19,desc,340302,72.981546
28,mths_since_last_delinq,250351,53.690554
47,next_pymnt_d,227214,48.728567
70,total_rev_hi_lim,70276,15.071469
58,tot_cur_bal,70276,15.071469
57,tot_coll_amt,70276,15.071469
10,emp_title,27588,5.916553
11,emp_length,21008,4.505399


### TOP 5 missing columns

| Table Name | Table Description | Missing percentage |
| --- | --- | --- |
| mths_since_last_record | The number of months since the last public record. | 84.6% |
| mths_since_last_major_derog | Months since most recent 90-day or worse rating. | 78.8% |
| desc | Loan description provided by the borrower. | 73% |
| mths_since_last_delinq | The number of months since the borrower's last delinquency. | 53.7% |
| next_pymnt_d | Next scheduled payment date | 48.7% |


<b> Hypothesis based on description </b>


- <b>mths_since_last_record</b> \
No activity/input of public record for the user 


- <b>mths_since_last_major_derog</b>\
No input/record for derogatory of the user


- <b>desc</b>\
User give no description for their loans


- <b>mths_since_last_delinq</b>\
No input/record for delinquency of the user


- <b>next_pymnt_d</b>\
No more payments needed (Check 'loan_status' column)